In [ ]:
import datetime

import pandas as pd
import numpy as np

from tqdm import tqdm
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
import glob
glob.glob("data09/*")

In [ ]:
train = pd.read_csv('data09\\funda_train.csv')

### 1. 데이터 전처리

#### 1-1. 자료형 변경 

In [ ]:
# 자료분포확인
train

In [ ]:
train.info()

일단, 불필요하게 메모리를 많이차지하는 컬럼들부터 자료형변환

In [ ]:
train.store_id = train.store_id.astype(np.int32)
train.card_id = train.card_id.astype(np.int32)
train.card_company = train.card_company.astype('category')
train.installment_term = train.installment_term.astype(np.int16)
train.region = train.region.astype('category')
train.type_of_business = train.type_of_business.astype('category')

#### 1-2. null 값 확인

In [ ]:
train.isnull().sum()

region과 type_of_business 은 null 값이 많고 변별력이 없는 것으로 판단 -> drop

In [ ]:
train = train.drop('region', axis=1)
train = train.drop('type_of_business', axis=1)
train

#### 1-3. 일자와 시간 컬럼을 병합하여 새로운 컬럼생성

In [ ]:
train['datetime'] = pd.to_datetime(train.transacted_date + " " + train.transacted_time, format='%Y-%m-%d %H:%M:%S')

#### 1-4. 환불값 제거

자료를 봤을 때 amount 중에 음수인 값이 있으므로 추측하건대 직원실수로 인한 환불이거나, 단순 변심으로 인한 환불로 추정됨.  
따라서 환불값은 제거할 필요가 있다고 판단

In [ ]:
# https://dacon.io/competitions/official/140472/codeshare/953?page=1&dtype=recent&ptype=pub
# 초보자라 환불값 제거하는 코드는 그대로 인용했습니다. 스스로 이해해볼려고 노력했습니다.

def refund_remove(df):
    refund=df[df['amount']<0] # 금액이 - 인 데이터프레임추출
    non_refund=df[df['amount']>0] # 금액이 + 인 데이터프레임 추출
    remove_data=pd.DataFrame() # 빈 DataFrame 생성
    
    for i in tqdm(df.store_id.unique()):
        divided_data=non_refund[non_refund['store_id']==i] ##non_refund 데이터를 상점별로 나눔
        divided_data2=refund[refund['store_id']==i] ##refund 데이터를 나눔 상점별로 나눔
        
        for neg in divided_data2.to_records()[:]: ##환불데이터를 차례대로 검사
            refund_store=neg['store_id']
            refund_id=neg['card_id'] ## 환불 카드 아이디
            refund_datetime=neg['datetime'] ## 환불 시간
            refund_amount=abs(neg['amount']) ## 환불액 절대값을 씌움
                
            ##환불시간 이전의 데이터중 카드이이디와 환불액이 같은 후보 리스트를 뽑는다.
            refund_pay_list=divided_data[divided_data['datetime']<=refund_datetime] # 환불시기가 구입시기이후 이어야하며
            refund_pay_list=refund_pay_list[refund_pay_list['card_id']==refund_id] # 환불 카드 아이디가 같아야하며
            refund_pay_list=refund_pay_list[refund_pay_list['amount']==refund_amount] # 환불금액과 구입금액이 같아야한다
                
                
            #후보리스트가 있으면,카드아이디, 환불액이 같으면서 가장 최근시간을 제거
            if(len(refund_pay_list)!=0): # 만약 환불내역이 있으면
                refund_datetime=max(refund_pay_list['datetime']) ##가장 최근 시간을 구한다
                remove=divided_data[divided_data['datetime']==refund_datetime] ##가장 최근시간
                remove=remove[remove['card_id']==refund_id] ##환불 카드 아이디
                remove=remove[remove['amount']==refund_amount] ##환불액
                divided_data=divided_data.drop(index=remove.index) #인덱스를 통해 제거
                    
        ##제거한데이터를 데이터프레임에 추가한다.
        remove_data=pd.concat([remove_data,divided_data],axis=0)
    
    return remove_data



In [ ]:
remove_df = refund_remove(train) # 초보자가 생각했을 때에는 오래걸리는 것 같은데 당연히 오래 걸리는 것인지 궁금합니다.

#### 1-4. 필요없는 컬럼제거

In [ ]:
del remove_df['transacted_date']
del remove_df['transacted_time']

#### 1-5. datetime 으로 index 설정

In [ ]:
remove_df.set_index('datetime', inplace = True)
remove_df

#### 1-6. 월별로 상점별 합계 구하기(최종 전처리된 데이터프레임생성)

In [ ]:
# https://dacon.io/competitions/official/140472/codeshare/953?page=1&dtype=recent&ptype=pub
# 초보자라 downsapling 코드 또한 그대로 인용했습니다. 스스로 이해해볼려고 노력했습니다.

data_month=pd.DataFrame() # 빈 DataFrame 생성
for i in tqdm(remove_df.store_id.unique()):
    data_num=remove_df[remove_df.store_id==i] # 상점별로 DataFrame 생성
    sum_amount=data_num['amount'].resample(rule='m').sum() # m별로 amount 합계를 구함
    data_mon=pd.concat([sum_amount],axis=1) # 잘 이해가 되지 않습니다.
    data_mon.insert(0,'store_id',i) # 잘 이해가 되지 않습니다
    data_month=pd.concat([data_month,data_mon],axis=0) # 아까만든 빈 DateFrame 과 행기준으로 병합

In [ ]:
data_month

### 2. 모델링 _지수이동평균

#### 2-1. 모델링

In [ ]:
# https://dacon.io/competitions/official/140472/codeshare/1374?page=1&dtype=recent&ptype=pub
# 지수이동평균코드 인용했습니다. 이해할려고 노력해봤습니다.

def sub(df,n):
    concat_3mon = pd.DataFrame(index=pd.to_datetime(['2019-03-31', '2019-04-30', '2019-05-31'])) # 최근3개월 매출을 기록할 DataFrame 생성
    df_ema=pd.DataFrame(columns=['store_id','amount']) # 컬럼명 생성
    ema_sub = np.array([]) # 빈 ndarray 생성

    for i in df.store_id.unique():
        df_set = pd.DataFrame(df[df.store_id == i].amount) # 상점별 amount 를 갖는 DataFrame 생성
        wma_train = pd.concat([df_set, concat_3mon], axis=0) # 아까만든 최근3개월 매출을 기록할 DataFrame과 df_set 을 병합
        num_sub = wma_train.amount.ewm(span=n).mean()['2019-03':].sum() # 관측데이터수를 n 개로하여 평균을 구함. 여기서 mean()['2019-03':].sum() 이 부분은 이해가 잘 안됩니다.
        df_ema.loc[i,'store_id']=i # store_id 컬럼만들고
        df_ema.loc[i,'amount']=num_sub # amount 컬럼만들기
        
    return df_ema

#### 2-2. 모델링 3개를 해서 평균을 낸 최종 예측결과 만들기

In [ ]:
ema_sub4 = sub(data_month, 4)
ema_sub5 = sub(data_month, 5)
ema_sub6 = sub(data_month, 6) 

In [ ]:
ema = ema_sub4 + ema_sub5 + ema_sub6 # 벡터화하면 빠르다해서 이렇게 만들어봤는데 이게 벡터화인지는 잘 모르겠습니다.
ema = ema / 3
ema

### 3-1. submisson 

In [ ]:
submit = pd.read_csv('data09\\submission.csv')
submit = submit[['store_id']].merge(ema, on='store_id', how = 'left')
submit.amount = submit.amount.fillna(0)

submit.to_csv('data09\\submission.csv', index = False)